In [1]:
import sys
sys.path.append('../..')

import pandas as pd
import datetime as dt
import numpy as np

from src.run_regression import run_cox_regression, run_gee_logistic_regression, export_cox_result, export_logistic_result
from sklearn.metrics import precision_recall_curve

In [9]:
results = []

In [14]:
for T_e in [3,5,7]:
    for T_a in [3,5,7]:
        cox, avg_cp = run_cox_regression(semester="fa21", T_e=T_e, T_a=T_a)
        res = export_cox_result(cox)
        coef = res["coeff"]["class_positivity"]
        hazard = [0] + [cox.baseline_cumulative_hazard_.iloc[2]["baseline hazard"]] + \
            list(np.diff(cox.baseline_cumulative_hazard_.reindex(
                list(range(cox.baseline_cumulative_hazard_.index.min(),cox.baseline_cumulative_hazard_.index.max()+1)),method="ffill").values.squeeze(1)
            ))
        avg_hazard = np.mean(hazard)

        base_risk = 1-(1-avg_hazard)**104
        risk_class = 1-(1-avg_hazard*np.exp(coef*avg_cp))**104

        results.append({
            "T_e": T_e,
            "T_a": T_a,
            "avg_cp": avg_cp,
            "coef": coef,
            "p_value": res["p-value"]["class_positivity"],
            "avg_hazard": avg_hazard,
            "base_risk": base_risk,
            "risk_class": risk_class,
            "risk_ratio": risk_class/base_risk,
            "risk_increase": risk_class-base_risk
        })

        res.to_csv(f"../../results/SA_cox_fa21_T_e={T_e}_T_a={T_a}.csv", float_format="%.2f")

c:\Program Files\Python37\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
../..\src\run_regression.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_idx'] = df['day_idx'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d').toordinal() - baseline_date)


Iteration 1: norm_delta = 8.21e-02, step_size = 0.1000, log_lik = -5909.25482, newton_decrement = 2.43e+02, seconds_since_start = 1.8
Iteration 2: norm_delta = 6.29e-02, step_size = 0.1000, log_lik = -5863.28890, newton_decrement = 1.49e+02, seconds_since_start = 3.5
Iteration 3: norm_delta = 5.17e-02, step_size = 0.1000, log_lik = -5835.07772, newton_decrement = 1.05e+02, seconds_since_start = 5.3
Iteration 4: norm_delta = 5.70e-02, step_size = 0.1300, log_lik = -5815.16124, newton_decrement = 7.80e+01, seconds_since_start = 7.1
Iteration 5: norm_delta = 4.61e-02, step_size = 0.1274, log_lik = -5796.24006, newton_decrement = 5.45e+01, seconds_since_start = 8.9
Iteration 6: norm_delta = 3.83e-02, step_size = 0.1249, log_lik = -5783.23969, newton_decrement = 3.96e+01, seconds_since_start = 10.6
Iteration 7: norm_delta = 4.29e-02, step_size = 0.1623, log_lik = -5773.98205, newton_decrement = 2.94e+01, seconds_since_start = 12.4
Iteration 8: norm_delta = 3.50e-02, step_size = 0.1591, log_

<lifelines.CoxTimeVaryingFitter: fitted with 1771641 periods, 17479 subjects, 606 events>
         event col = 'infected_on_this_day'
number of subjects = 17479
 number of periods = 1771641
  number of events = 606
partial log-likelihood = -5745.18
  time fit was run = 2024-05-09 20:07:39 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
class_positivity       2.61      13.61       0.34             1.95             3.27                 7.03                26.34
academic_career_GM     0.14       1.15       0.27            -0.38             0.66                 0.68                 1.94
academic_career_GR    -0.97       0.38       0.41            -1.78            -0.16                 0.17                 0.85
academic_career_LA     0.55       1.73       0.20             0.15             0.94                 1.16                 2.57
academic_career_UG_A   0.73       2.08       0.16             0.42             1.05                 1.52                 2.86
academic_career_UG_G   1.48       4.41       0.09             1.31             1.66                 3.70                 5.26
academic_career_VM     0.22       1.24       0.31            -0.39             0.82                 0.68                 2.28
current_gender_M       0.06       1.06       0.08            -0.10             0.22                 0.91                 1.25

                       cmp to     z      p   -log2(p)
covariate                                            
class_positivity         0.00  7.75 <0.005      46.62
academic_career_GM       0.00  0.52   0.60       0.73
academic_career_GR       0.00 -2.34   0.02       5.69
academic_career_LA       0.00  2.69   0.01       7.14
academic_career_UG_A     0.00  4.56 <0.005      17.58
academic_career_UG_G     0.00 16.53 <0.005     201.38
academic_career_VM       0.00  0.70   0.48       1.06
current_gender_M         0.00  0.75   0.46       1.13
---
Partial AIC = 11506.35
log-likelihood ratio test = 328.16 on 8 df
-log2(p) of ll-ratio test = 217.20

c:\Program Files\Python37\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
../..\src\run_regression.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_idx'] = df['day_idx'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d').toordinal() - baseline_date)


Iteration 1: norm_delta = 8.27e-02, step_size = 0.1000, log_lik = -5909.25482, newton_decrement = 2.49e+02, seconds_since_start = 1.8
Iteration 2: norm_delta = 6.29e-02, step_size = 0.1000, log_lik = -5862.13278, newton_decrement = 1.50e+02, seconds_since_start = 3.6
Iteration 3: norm_delta = 5.17e-02, step_size = 0.1000, log_lik = -5833.69790, newton_decrement = 1.05e+02, seconds_since_start = 5.3
Iteration 4: norm_delta = 5.70e-02, step_size = 0.1300, log_lik = -5813.70257, newton_decrement = 7.82e+01, seconds_since_start = 7.1
Iteration 5: norm_delta = 4.61e-02, step_size = 0.1274, log_lik = -5794.73502, newton_decrement = 5.46e+01, seconds_since_start = 8.9
Iteration 6: norm_delta = 3.82e-02, step_size = 0.1249, log_lik = -5781.71617, newton_decrement = 3.96e+01, seconds_since_start = 10.6
Iteration 7: norm_delta = 4.29e-02, step_size = 0.1623, log_lik = -5772.45017, newton_decrement = 2.94e+01, seconds_since_start = 12.3
Iteration 8: norm_delta = 3.50e-02, step_size = 0.1591, log_

...

Iteration 1: norm_delta = 8.38e-02, step_size = 0.1000, log_lik = -5909.25482, newton_decrement = 2.39e+02, seconds_since_start = 1.7
Iteration 2: norm_delta = 6.38e-02, step_size = 0.1000, log_lik = -5863.95962, newton_decrement = 1.54e+02, seconds_since_start = 3.5
Iteration 3: norm_delta = 5.20e-02, step_size = 0.1000, log_lik = -5834.71688, newton_decrement = 1.08e+02, seconds_since_start = 5.3
Iteration 4: norm_delta = 5.70e-02, step_size = 0.1300, log_lik = -5814.16928, newton_decrement = 7.99e+01, seconds_since_start = 7.0
Iteration 5: norm_delta = 4.60e-02, step_size = 0.1274, log_lik = -5794.77269, newton_decrement = 5.56e+01, seconds_since_start = 8.8
Iteration 6: norm_delta = 3.82e-02, step_size = 0.1249, log_lik = -5781.52630, newton_decrement = 4.02e+01, seconds_since_start = 10.5
Iteration 7: norm_delta = 4.28e-02, step_size = 0.1623, log_lik = -5772.12781, newton_decrement = 2.98e+01, seconds_since_start = 12.3
Iteration 8: norm_delta = 3.49e-02, step_size = 0.1591, log_

<lifelines.CoxTimeVaryingFitter: fitted with 1771641 periods, 17479 subjects, 606 events>
         event col = 'infected_on_this_day'
number of subjects = 17479
 number of periods = 1771641
  number of events = 606
partial log-likelihood = -5743.05
  time fit was run = 2024-05-09 20:16:33 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
class_positivity       1.90       6.69       0.24             1.44             2.36                 4.20                10.63
academic_career_GM     0.16       1.17       0.27            -0.36             0.68                 0.69                 1.97
academic_career_GR    -0.94       0.39       0.41            -1.75            -0.13                 0.17                 0.88
academic_career_LA     0.56       1.75       0.20             0.16             0.96                 1.18                 2.61
academic_career_UG_A   0.73       2.08       0.16             0.41             1.05                 1.51                 2.85
academic_career_UG_G   1.48       4.38       0.09             1.30             1.65                 3.68                 5.23
academic_career_VM     0.24       1.27       0.31            -0.37             0.85                 0.69                 2.33
current_gender_M       0.06       1.06       0.08            -0.10             0.22                 0.91                 1.25

                       cmp to     z      p   -log2(p)
covariate                                            
class_positivity         0.00  8.03 <0.005      49.84
academic_career_GM       0.00  0.59   0.55       0.85
academic_career_GR       0.00 -2.28   0.02       5.47
academic_career_LA       0.00  2.77   0.01       7.48
academic_career_UG_A     0.00  4.53 <0.005      17.39
academic_career_UG_G     0.00 16.44 <0.005     199.23
academic_career_VM       0.00  0.78   0.44       1.19
current_gender_M         0.00  0.75   0.45       1.14
---
Partial AIC = 11502.09
log-likelihood ratio test = 332.42 on 8 df
-log2(p) of ll-ratio test = 220.22

c:\Program Files\Python37\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
../..\src\run_regression.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_idx'] = df['day_idx'].apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d').toordinal() - baseline_date)


Iteration 1: norm_delta = 8.48e-02, step_size = 0.1000, log_lik = -5909.25482, newton_decrement = 2.45e+02, seconds_since_start = 1.8
Iteration 2: norm_delta = 6.44e-02, step_size = 0.1000, log_lik = -5862.85619, newton_decrement = 1.59e+02, seconds_since_start = 3.5
Iteration 3: norm_delta = 5.22e-02, step_size = 0.1000, log_lik = -5832.77744, newton_decrement = 1.11e+02, seconds_since_start = 5.3
Iteration 4: norm_delta = 5.72e-02, step_size = 0.1300, log_lik = -5811.79993, newton_decrement = 8.12e+01, seconds_since_start = 7.0
Iteration 5: norm_delta = 4.62e-02, step_size = 0.1274, log_lik = -5792.10404, newton_decrement = 5.62e+01, seconds_since_start = 8.8
Iteration 6: norm_delta = 3.83e-02, step_size = 0.1249, log_lik = -5778.70427, newton_decrement = 4.06e+01, seconds_since_start = 10.6
Iteration 7: norm_delta = 4.29e-02, step_size = 0.1623, log_lik = -5769.21163, newton_decrement = 3.00e+01, seconds_since_start = 12.4
Iteration 8: norm_delta = 3.50e-02, step_size = 0.1591, log_

<lifelines.CoxTimeVaryingFitter: fitted with 1771641 periods, 17479 subjects, 606 events>
         event col = 'infected_on_this_day'
number of subjects = 17479
 number of periods = 1771641
  number of events = 606
partial log-likelihood = -5739.88
  time fit was run = 2024-05-09 20:17:48 UTC

---
                       coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                    
class_positivity       1.86       6.39       0.20             1.47             2.24                 4.33                 9.44
academic_career_GM     0.16       1.17       0.27            -0.37             0.68                 0.69                 1.97
academic_career_GR    -0.93       0.39       0.41            -1.74            -0.12                 0.18                 0.89
academic_career_LA     0.57       1.77       0.20             0.17             0.97                 1.19                 2.63
academic_career_UG_A   0.73       2.08       0.16             0.42             1.05                 1.52                 2.85
academic_career_UG_G   1.48       4.39       0.09             1.30             1.66                 3.69                 5.24
academic_career_VM     0.25       1.28       0.31            -0.36             0.86                 0.70                 2.35
current_gender_M       0.06       1.07       0.08            -0.10             0.22                 0.91                 1.25

                       cmp to     z      p   -log2(p)
covariate                                            
class_positivity         0.00  9.34 <0.005      66.47
academic_career_GM       0.00  0.59   0.55       0.85
academic_career_GR       0.00 -2.25   0.02       5.36
academic_career_LA       0.00  2.81 <0.005       7.65
academic_career_UG_A     0.00  4.55 <0.005      17.48
academic_career_UG_G     0.00 16.51 <0.005     200.92
academic_career_VM       0.00  0.81   0.42       1.26
current_gender_M         0.00  0.77   0.44       1.18
---
Partial AIC = 11495.76
log-likelihood ratio test = 338.75 on 8 df
-log2(p) of ll-ratio test = 224.70

In [13]:
np.diff(cox.baseline_cumulative_hazard_["baseline hazard"].values).mean()

0.0003775202603782674

In [18]:
pd.DataFrame(results).to_csv("../../results/SA_cox_fa21_summary.csv", float_format="%.4f")